# Libraries

In [48]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [49]:
# Get the current user's name
user = getuser()

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\euro_goals.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,stage,time,home_team,away_team,score,stadium_name,stadium_city,stadium_attendance,referee_name,referee_nationality,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
0,Group 1,20:30,France,Denmark,1–0,Parc des Princes,Paris,47570,Volker Roth,West Germany,...,78,0,1,0,0,0,0,0,1984-06-12,12 June 1984
1,Group 1,20:30,Belgium,Yugoslavia,2–0,Stade Félix-Bollaert,Lens,41525,Erik Fredriksson,Sweden,...,28,0,2,0,0,0,0,0,1984-06-13,13 June 1984
2,Group 1,20:30,Belgium,Yugoslavia,2–0,Stade Félix-Bollaert,Lens,41525,Erik Fredriksson,Sweden,...,45,0,2,0,0,0,0,0,1984-06-13,13 June 1984
3,Group 1,17:15,France,Belgium,5–0,Stade de la Beaujoire,Nantes,51359,Bob Valentine,Scotland,...,4,0,5,0,0,0,0,0,1984-06-16,16 June 1984
4,Group 1,17:15,France,Belgium,5–0,Stade de la Beaujoire,Nantes,51359,Bob Valentine,Scotland,...,74,0,5,0,0,1,0,0,1984-06-16,16 June 1984


# Clean and transfrom variables

## time

In [50]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


## date

In [51]:
# Convert 'short_date' column to datetime if it's not already
df['short_date'] = pd.to_datetime(df['short_date'])

# Extract the year from 'short_date' and create a new column 'year'
df['year'] = df['short_date'].dt.year

# Extract relevant columns

In [52]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final'])]

# Extract relevant columns for goal events and match results
goals_df = df[['year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time']]

display(goals_df.head())


,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time
0,1984,Group 1,France,Denmark,France,78,1984-06-12,20:30
1,1984,Group 1,Belgium,Yugoslavia,Belgium,28,1984-06-13,20:30
2,1984,Group 1,Belgium,Yugoslavia,Belgium,45,1984-06-13,20:30
3,1984,Group 1,France,Belgium,France,4,1984-06-16,17:15
4,1984,Group 1,France,Belgium,France,74,1984-06-16,17:15


# Recreate Leauge Table after first two matchdays

### Filter out the matches from the last match day in the goals_df DataFrame.

In [53]:
# Step 1: Find the last match date for each tournament and group
last_dates = goals_df.groupby(['year', 'stage'])['short_date'].max().reset_index()

# Step 2: Create the dataset with all games excluding the last match day (goals_before_last_day)
# Performing a left merge and keeping only rows not present in the last_dates
goals_before_last_day = goals_df.merge(last_dates, on=['year', 'stage', 'short_date'], how='left', indicator=True)
goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 3: Create the dataset with only the last match day games (goals_last_day)
goals_last_day = goals_df.merge(last_dates, on=['year', 'stage', 'short_date'], how='inner')

# Step 4: Sort the DataFrame by 'goal_minute'
goals_last_day_sorted = goals_last_day.sort_values(by=['goal_minute'], ascending=True)


## create df  of aggregate data for each match